In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
path_name="C:\\Users\\Home\\CIP_Lab Related Works\\DataSet1\\Amazon_Product.csv"
df=pd.read_csv(path_name,encoding= 'unicode_escape')

In [3]:
df.dropna(subset=["Rating"] , inplace = True)
df["Rating"] = df["Rating"].astype(int)

In [4]:
df.shape

(413840, 6)

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
for train_index, test_index in split.split(df, df["Rating"]): 
    strat_train = df.reindex(train_index)
    strat_test = df.reindex(test_index)

In [6]:
X_train = strat_train["Reviews"]
#X_train_targetSentiment = strat_train["reviews.rating"].astype(int)
X_test = strat_test["Reviews"]
#X_test_targetSentiment = strat_test["reviews.rating"].astype(int)
print(len(X_train), len(X_test))
#print(len(X_train_targetSentiment), len(X_test_targetSentiment))

331072 82768


In [7]:
 from tensorflow import keras
model = keras.models.load_model('./Amazon_Final.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
from sklearn.utils import shuffle
data_majority = df[df['Rating'] <=3]
data_minority = df[df['Rating'] >=4]

bias = data_minority.shape[0]/data_majority.shape[0]
# lets split train/test data first then 
train = pd.concat([data_majority.sample(frac=0.6,random_state=200),
         data_minority.sample(frac=0.6,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.6,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.6,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

In [9]:
print('positive data in training:',(train['Rating'] >=4).sum())
print('negative data in training:',(train['Rating'] <=3).sum())
print('positive data in test:',(test['Rating'] >=4).sum())
print('negative data in test:',(test['Rating'] <=3).sum())

positive data in training: 170998
negative data in training: 77306
positive data in test: 113999
negative data in test: 51537


In [10]:
print(df['Rating'].unique())
data_majority = train[train['Rating'] >=4]
data_minority = train[train['Rating'] <=3]

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)
from sklearn.utils import resample
# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=42) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
print("After upsampling\n",data_upsampled['Rating'].value_counts(),sep = "")


[5 4 1 2 3]
majority class before upsample: (170998, 6)
minority class before upsample: (77306, 6)
After upsampling
5    134017
1     96003
3     42471
4     36981
2     32524
Name: Rating, dtype: int64


In [11]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Reviews'].astype(str).values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['Reviews'].astype(str).values)
X_train = pad_sequences(X_train,maxlen=29)
Y_train = pd.get_dummies(data_upsampled['Rating']).astype(str).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['Reviews'].astype(str).values)
X_test = pad_sequences(X_test,maxlen=29)
Y_test = pd.get_dummies(test['Rating']).values
print("x_test shape", X_test.shape)

x_train shape: (341996, 29)
x_test shape (165536, 29)


In [12]:
import numpy as np
batch_size = 128
from sklearn.metrics import confusion_matrix,classification_report
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[ 7385  2557  2007  2767 14279]
 [ 2510   873   842  1107  4676]
 [ 2624  1024   982  1424  6480]
 [ 4931  1723  1535  2495 13727]
 [18944  5120  4239  7050 54235]]
              precision    recall  f1-score   support

           0       0.20      0.25      0.23     28995
           1       0.08      0.09      0.08     10008
           2       0.10      0.08      0.09     12534
           3       0.17      0.10      0.13     24411
           4       0.58      0.61      0.59     89588

    accuracy                           0.40    165536
   macro avg       0.23      0.23      0.22    165536
weighted avg       0.39      0.40      0.39    165536



In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Reviews'].astype(str).values)
twt = ["Product is  damaged"]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
#print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)

1/1 - 0s
[9.9551618e-01 4.1723251e-07 2.7716160e-06 3.5762787e-06 5.6379715e-08]


In [14]:
import numpy as np
i = np.argmax(sentiment)
print(np.argmax(sentiment))
if i<=1:
        flag="Negative"
        print("Negative")
if i==2:
        flag="Neutral"
        print("Neural")
if(i>=3):
        flag="Positive"
        print("Positive")

0
Negative


In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Reviews'].astype(str).values)
twt = ["package superb"]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
#print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)

1/1 - 0s
[7.7784061e-06 2.6154518e-04 5.5521727e-05 5.4647368e-01 7.0978171e-04]


In [16]:
import numpy as np
i = np.argmax(sentiment)
print(np.argmax(sentiment))
if i<=1:
        flag="Negative"
        print("Negative")
if i==2:
        flag="Neutral"
        print("Neural")
if(i>=3):
        flag="Positive"
        print("Positive")

3
Positive


In [13]:
from flask import Flask, render_template, request
global model1
from tensorflow.keras.models import Sequential, load_model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import h5py
app = Flask(__name__,static_folder='C:\\Users\\Home\\Amazon Sentiment Analysis\\static')
@app.route('/')
def index_view():
    return render_template('temp.html')

def get_model():
    global model1
    #model = Sequential()
        # this is key : save the graph after loading the model
    global graph
    graph = tf.get_default_graph()


# Initialization

    model1 = keras.models.load_model('./Amazon_Final.h5')
    #model=load_model("AmazonReview.h5")
    print(" * Model loaded!")

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# max_fatures = 2000
# tokenizer = Tokenizer(num_words=max_fatures, split=' ')
# tokenizer.fit_on_texts(df['Reviews'].astype(str).values)
# twt = ["Product is good but package damaged"]
# #vectorizing the tweet by the pre-fitted tokenizer instance
# twt = tokenizer.texts_to_sequences(twt)
# #padding the tweet to have exactly the same shape as `embedding_2` input
# twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
# #print(twt)
# sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
# print(sentiment)


# @app.route('/predict',methods=['GET','POST'])
@app.route('/', methods=['GET','POST'])
def predict(): 
    text1 = request.form['text1']
    max_fatures = 2000
    #tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    #text_final = ''.join(c for c in text1 if not c.isdigit())
    #tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    #tokenizer.fit_on_texts(df['Reviews'].astype(str).values)
    twt1 = text1
    #vectorizing the tweet by the pre-fitted tokenizer instance
    twt1 = tokenizer.texts_to_sequences(twt1)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    twt1 = pad_sequences(twt1, maxlen=29, dtype='int32', value=0)
    sentiment = model1.predict(twt1,batch_size=1,verbose = 2)[0]
    print(sentiment)
    i = np.argmax(sentiment)
    print(np.argmax(sentiment))
    if i<=1:
        flag="Negative"
        print("Negative")
    if i==2:
        flag="Neutral"
        print("Neural")
    if(i>=3):
        flag="Positive"
        print("Positive")
    return render_template('index.html',name="User",positive=(sentiment[3]+sentiment[4]),neutral=sentiment[2],negative=(sentiment[0]+sentiment[1]))
    #return render_template('temp.html', final=sentiment[3]+sentiment[4], text1=text1,text2=sentiment[3]+sentiment[4],text3=sentiment[2],text5=sentiment[0]+sentiment[1],text4=flag)
# @app.route('/', methods=['POST'])
# def predict1(): 
#     text1 = request.form['text1']
#     #text_final = ''.join(c for c in text1 if not c.isdigit())
#     twt = text1
#     #vectorizing the tweet by the pre-fitted tokenizer instance
#     twt = tokenizer.texts_to_sequences(twt)
#     #padding the tweet to have exactly the same shape as `embedding_2` input
#     twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
#     print(twt)
#     sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
#     print(sentiment)
#     return render_template('temp.html', final=max(sentiment), text1=text1,text2=sentiment[2],text5=sentiment[0],text4=max(sentiment),text3=sentiment[1])

In [15]:
get_model()

 * Model loaded!


In [ ]:
if __name__ == '__main__':
    app.run(threaded=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2021 16:49:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2021 16:49:15] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2021 16:49:20] "GET /favicon.ico HTTP/1.1" 404 -


18/18 - 1s


127.0.0.1 - - [26/May/2021 16:49:35] "POST / HTTP/1.1" 200 -


[8.8114440e-03 2.9519200e-04 1.0842383e-03 8.6150169e-03 9.6922934e-01]
4
Positive


...

127.0.0.1 - - [26/May/2021 16:56:55] "GET / HTTP/1.1" 200 -


9/9 - 0s


127.0.0.1 - - [26/May/2021 16:57:05] "POST / HTTP/1.1" 200 -


[7.2558582e-01 8.7320805e-05 1.9669533e-06 1.0392070e-04 1.1725811e-05]
0
Negative


127.0.0.1 - - [26/May/2021 16:57:15] "GET / HTTP/1.1" 200 -


In [ ]:
#End

In [ ]:
model = keras.models.load_model('./Amazon_Final.h5')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Reviews'].astype(str).values)
twt = ["Product is  good"]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
#print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)

In [ ]:
import numpy as np
i = np.argmax(sentiment)
print(np.argmax(sentiment))
if i<=1:
        flag="Negative"
        print("Negative")
if i==2:
        flag="Neutral"
        print("Neural")
if(i>=3):
        flag="Positive"
        print("Positive")